# 0. Uso de módulos Perl

In [1]:
use strict;
use warnings;
use Data::Dump qw(dump);
use d2l;
use AI::MXNet qw(mx shape);
use List::MoreUtils qw(each_arrayref);

# 1. Lectura del archivo que contiene la salida del algoritmo Smith-Waterman

Procesamiento realizado a nivel de caracteres, esto con el fin de:
1. Colocar una secuencia de caracteres, es este caso "(-)" para posteriormente realizar la etiquetación de delimitador. Para este caso se procede a verificar los dos caracteres anteriores y los dos posteriores tanto si encuentra un espacio en ocr y un guión en ref o viceversa. Si entra en esta condición se agrega la secuencia mencionada si solo si los dos caracteres antes del guión son iguales tanto en ocr como en ref y viceversa, caso contrario si encuentra la condicion de que el caracter anterior o posterior es igual tanto en ocr como en ref se agrega un espacio en lugar del guión, esto para tener el mismo número de palabras en los casos de agregación ya que aquí tanto ocr como ref no tienen el mismo número de palabras por fila.

In [2]:
sub procesamiento_caracteres {
    my ($ocr_chars, $ref_chars) = @_;

    for my $i (0 .. scalar(@$ocr_chars) - 1) {
        if ((('-' eq $ocr_chars->[$i] && ' ' eq $ref_chars->[$i]) || (' ' eq $ocr_chars->[$i] && '-' eq $ref_chars->[$i])) && ($i > 1 && $i < scalar(@$ocr_chars) - 2)) {
            my $prev_ocr_char = $ocr_chars->[$i - 1];
            my $prev_ocr_char_2 = $ocr_chars->[$i - 2];
            my $next_ocr_char = $ocr_chars->[$i + 1];
            my $next_ocr_char_2 = $ocr_chars->[$i + 2];
            my $prev_ref_char = $ref_chars->[$i - 1];
            my $prev_ref_char_2 = $ref_chars->[$i - 2];
            my $next_ref_char = $ref_chars->[$i + 1];
            my $next_ref_char_2 = $ref_chars->[$i + 2];

            if (($prev_ocr_char eq $prev_ref_char && $next_ocr_char eq $next_ref_char) && ($prev_ocr_char_2 eq $prev_ref_char_2 && $next_ocr_char_2 eq $next_ref_char_2)) {
                $ocr_chars->[$i] = '(-)';
                $ref_chars->[$i] = '(-)';
            } elsif ($prev_ocr_char eq $prev_ref_char || $next_ocr_char eq $next_ref_char) {
                $ocr_chars->[$i] = ' ' if $ocr_chars->[$i] eq '-';
                $ref_chars->[$i] = ' ' if $ref_chars->[$i] eq '-';
            }
        }
    }

    return ($ocr_chars, $ref_chars);
}

Función que recibe como entrada la direccion donde esta el archivo que contiene el resultado de la aplicación del algoritmo Smith-Waterman (linea ocr seguido de linea ref seguido un espacio).
Este archivo es guardado en la variable $corpus inicialmente, para luego realizar el preprocesamiento para asignar las filas a @corpus_OCR y a @corpus_REF.
También, cuando retorna los resultados de esta función se imprime aquellas filas que no tienen el mismo número de filas y a estas filas se procede con la función de procesamiento_caracteres.

In [3]:
sub leer_archivo {
    my $nombre_archivo = shift;

    my $corpus;
    my @corpus_OCR;
    my @corpus_REF;

    open(my $archivo, '<:encoding(UTF-8)', $nombre_archivo) or die "No se pudo abrir el archivo '$nombre_archivo': $!";
    {
        local $/;  # Cambiar el separador de líneas para leer todo el archivo
        $corpus = <$archivo>;
    }
    close($archivo);

    my @filas = split /\n/, $corpus;

    @filas = grep { $_ ne '' } @filas; # Eliminar filas vacías

    # Separar filas impares y pares en OCR y REF
    for (my $i = 0; $i < @filas; $i++) {
        if ($i % 2 == 0) {
            push @corpus_OCR, $filas[$i];
        } else {
            push @corpus_REF, $filas[$i];
        }
    }

    # procesamiento a nivel de caracteres
    for my $i (0 .. $#corpus_OCR) {
        my @palabras_OCR = split /\s+/, $corpus_OCR[$i];
        my @palabras_REF = split /\s+/, $corpus_REF[$i];
        
        my @ocr_chars = split //, $corpus_OCR[$i];
        my @ref_chars = split //, $corpus_REF[$i];
        
        procesamiento_caracteres(\@ocr_chars, \@ref_chars);

        $corpus_OCR[$i] = join('', @ocr_chars);
        $corpus_REF[$i] = join('', @ref_chars);
    }
    
    return (\@corpus_OCR, \@corpus_REF);
}

Uso de la función leer_archivo y comprobación del tamaño de las variables corpus_ocr y corpus_ref para comprobar que tienen el mismo número de filas

In [4]:
my $nombre_archivo = "../../TesisModelo1/0_Corpus/2_Entrenamiento/Smith-Waterman/corpus_SW.txt";
my ($corpus_OCR, $corpus_REF) = leer_archivo($nombre_archivo);

my $numero_filas_OCR = scalar(@{$corpus_OCR});
my $numero_filas_REF = scalar(@{$corpus_REF});

print "Número de filas en OCR: $numero_filas_OCR\nNúmero de filas en REF: $numero_filas_REF\n";

Número de filas en OCR: 237097
Número de filas en REF: 237097


1

La función verificar_numero_palabras, comprueba que se tenga el mismo número de palabras en cada fila tanto en ocr como en ref.

In [5]:
sub verificar_numero_palabras {
    my ($corpus_OCR, $corpus_REF) = @_;

    my @lineas_diferentes;

    for (my $i = 0; $i < @$corpus_OCR; $i++) {
        my @palabras_OCR = split /\s+/, $corpus_OCR->[$i];
        my @palabras_REF = split /\s+/, $corpus_REF->[$i];

        if (@palabras_OCR != @palabras_REF) {
            push @lineas_diferentes, $i + 1;
            print "Fila $i:\n";
            print "OCR: " . $corpus_OCR->[$i] . "\n";
            print "REF: " . $corpus_REF->[$i] . "\n";
        }
    }
    
    if(!@lineas_diferentes) {
        print "Todas las líneas tienen el mismo número de palabras!!";
    }

    return @lineas_diferentes;
}

In [6]:
my @lineas_diferentes = verificar_numero_palabras($corpus_OCR, $corpus_REF);

Fila 16:
OCR: oc kernel patcher skips a a patch at u due to version u u u--
REF: oc kernel patcher skips a a patch at u due to version u u u n
Fila 28:
OCR: it generates primes till a given number m
REF: it generates primes till a given number--
Fila 216:
OCR: 3 using bouncing loader div we target- the three child div s of the parent to be styled the div s are given a width and height of 1rem using border radius 50 to turn them from squares to circles
REF: 3------ bouncing loader div -- targets the three child div s of the parent to be styled the div s are given a width and height of 1rem using border radius 50 to turn them from squares to circles
Fila 290:
OCR: a state-ful component is a component whose behavior does not depend on its state state-ful components can be either functional or class components stateless functional components are easier to maintain and test while they avoid the this keyword altogether stateless ---c---lass components should be preferred when lifecycle hooks

Como aún se tiene filas que no tienen el mismo número de palabras, se procede a comparar caracteres, esto con el fin de comparar guiones y espacios en la misma posicón en cada fila, si encuentra esta condición elimina el guión y coloca un espacio, esto con el fin de igualar a la otra fila si es de ocr o de ref.

In [7]:
sub comparar_caracteres {
    my ($corpus_OCR, $corpus_REF) = @_;

    my @corpus_OCR_modificado;
    my @corpus_REF_modificado;

    for (my $i = 0; $i < @$corpus_OCR; $i++) {
        my @ocr_chars = split //, $corpus_OCR->[$i];
        my @ref_chars = split //, $corpus_REF->[$i];
        
        my @palabras_OCR = split /\s+/, $corpus_OCR->[$i];
        my @palabras_REF = split /\s+/, $corpus_REF->[$i];
        
        if (@palabras_OCR != @palabras_REF) {
            for (my $j = 0; $j < @ocr_chars; $j++) {
                if ($ocr_chars[$j] eq '-' && $ref_chars[$j] eq ' ') {
                    $ocr_chars[$j] = ' ';
                } elsif ($ocr_chars[$j] eq ' ' && $ref_chars[$j] eq '-') {
                    $ref_chars[$j] = ' ';
                }
            }
        }

        push @corpus_OCR_modificado, join('', @ocr_chars);
        push @corpus_REF_modificado, join('', @ref_chars);
    }

    return (\@corpus_OCR_modificado, \@corpus_REF_modificado);
}

In [8]:
($corpus_OCR, $corpus_REF) = comparar_caracteres($corpus_OCR, $corpus_REF);

ARRAY(0x128d2a30)ARRAY(0x128d2a48)

Verificar si aún se encuentran filas con distinto número de palabras.

In [9]:
my @lineas_diferentes_2 = verificar_numero_palabras($corpus_OCR, $corpus_REF);

Fila 1330:
OCR: along the way we re going to explore software that will help you understand if your customers are excited sad or disgusted by your products we ll see how simple api calls can identify what objects are in an image we ll show how to build your own iphone app that cal tell you whether what you re pointing your camera at is a business card
REF: along the way we re -di--scove----r- software that will help you understand if your customers are excited sad or disgusted by your products we ll see how simple api calls can identify what objects are in an image we ll show how to build your own iphone app that can tell you whether ---- you re pointing your camera at -- a business card
Fila 1951:
OCR: throughout the rest of this guide we will demonstrate the power of the enterprise row model with the aid a j ava
REF: throughout the rest of this guide we will demonstrate the power of the enterprise row model with the aid -of a--
Fila 2132:
OCR: when using optimisticreponse helper from

Finalmente se observa que se tiene distinto número de filas debido a la falta de guiones, pero esto no tiene que ver con la función comparar_caracteres, ya que aqui eliminaba aquellos donde encontraba un espacio y un guión en la misma posición de la dos filas.
En este caso se procede a verificar si en una fila hay un guión y en la otra fila se encuentra un caracter alfanumérico para poder agregar un guión e igualar el número de palabras.

In [10]:
sub ajustar_numero_palabras {
    my ($corpus_OCR, $corpus_REF) = @_;

    my @corpus_OCR_modificado;
    my @corpus_REF_modificado;

    for (my $i = 0; $i < @$corpus_OCR; $i++) {
        my @ocr_chars = split //, $corpus_OCR->[$i];
        my @ref_chars = split //, $corpus_REF->[$i];
        
        my @palabras_OCR = split /\s+/, $corpus_OCR->[$i];
        my @palabras_REF = split /\s+/, $corpus_REF->[$i];
        
        if (@palabras_OCR != @palabras_REF) {
            for (my $j = 0; $j < @ocr_chars; $j++) {
                if ($ocr_chars[$j] eq ' ' && $ref_chars[$j] =~ /[a-zA-Z0-9]/) {
                    $ocr_chars[$j] = '-';
                } elsif ($ocr_chars[$j] =~ /[a-zA-Z0-9]/ && $ref_chars[$j] eq ' ') {
                    $ref_chars[$j] = '-';
                }
            }
        }
        
        push @corpus_OCR_modificado, join('', @ocr_chars);
        push @corpus_REF_modificado, join('', @ref_chars);
    }

    return (\@corpus_OCR_modificado, \@corpus_REF_modificado);
}

In [11]:
($corpus_OCR, $corpus_REF) = ajustar_numero_palabras($corpus_OCR, $corpus_REF);

ARRAY(0xd4c8790)ARRAY(0xd4c87c0)

In [12]:
my @lineas_diferentes_3 = verificar_numero_palabras($corpus_OCR, $corpus_REF);

Todas las líneas tienen el mismo número de palabras!!

In [13]:
sub oracion_mas_larga {
  my $corpus = shift;

  my $oracion_mas_larga = "";
  my $longitud_maxima = 0;

  foreach my $oracion (@{$corpus}) {
    my @palabras = split(' ', $oracion);
    my $longitud_oracion = scalar @palabras;

    if ($longitud_oracion > $longitud_maxima) {
      $longitud_maxima = $longitud_oracion;
      $oracion_mas_larga = $oracion;
    }
  }

  return $oracion_mas_larga, $longitud_maxima;
}

In [14]:
my ($oracion_mas_larga_OCR, $longitud_mas_larga_OCR) = oracion_mas_larga($corpus_OCR);
my ($oracion_mas_larga_REF, $longitud_mas_larga_REF) = oracion_mas_larga($corpus_REF);

print "La oración más larga en el archivo OCR es:\n$oracion_mas_larga_OCR\n";
print "Número de palabras: $longitud_mas_larga_OCR\n\n";

print "La oración más larga en el archivo REF es:\n$oracion_mas_larga_REF\n";
print "Número de palabras: $longitud_mas_larga_REF\n";

La oración más larga en el archivo OCR es:
assuming there are two msts called t and t for any edge e in t if we refove e from t then t becomes unconnected and we have a cut s v s according to exercise 23 1 3 e is the light edge through cut s v s if edge x is in t and through cut s v s then x is also a light weight because the light edge is unique so e and x is the same edge e is also in t because we choose e at random of all edges in t also in t as a result the mst is unique
Número de palabras: 108

La oración más larga en el archivo REF es:
assuming there are two msts called t and t for any edge e in t if we remove e from t then t becomes unconnected and we have a cut s v s according to exercise 23 1 3 e is the light edge through cut s v s if edge x is in t and through cut s v s then x is also a light weight because the light edge is unique so e and x is the same edge e is also in t because we choose e at random of all edges in t also in t as a result the mst is unique
Número de palab

1

In [15]:
open(my $archivo_OCR, '>:encoding(UTF-8)', '../../TesisModelo1/0_Corpus/3_Entrenamiento/corpus_OCR_procesada.txt') or die "No se puede abrir el archivo corpus_OCR_procesada.txt: $!";
foreach my $linea (@{$corpus_OCR}) {
    print $archivo_OCR $linea . "\n";
}
close($archivo_OCR);

open(my $archivo_REF, '>:encoding(UTF-8)', '../../TesisModelo1/0_Corpus/3_Entrenamiento/corpus_REF_procesada.txt') or die "No se puede abrir el archivo corpus_REF_procesada.txt: $!";
foreach my $linea (@{$corpus_REF}) {
    print $archivo_REF $linea . "\n";
}
close($archivo_REF);

1